In [ ]:
!pip install torch whisper gradio gtts deep-translator
!pip install langdetect
!pip install deep-translator
!pip install gtts
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.7 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=485cb84dc76cc3e4550fa7ee43f6a62cbf1de25e69c3378efe79204da2ba4b22
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a

In [ ]:
import cv2
from transformers import pipeline, AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import numpy as np

In [ ]:
# Load the processor and model
processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224")

# High-level pipeline for classification
pipe = pipeline("image-classification", model=model, feature_extractor=processor)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Use the pipeline for high-level operations
pipe1 = pipeline("text2text-generation", model="google/flan-t5-large")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("google/flan-t5-large")
model1 = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

In [ ]:
def get_brief_description(word):
    """
    Generate a brief explanation of a word using the flan-t5-large model.
    Args:
    - word (str): The word to explain.

    Returns:
    - str: The model-generated description.
    """
    prompt = f"explain what is  {word}"

    # Encode the prompt and generate the explanation
    inputs = tokenizer1(prompt, return_tensors="pt")
    outputs = model1.generate(inputs["input_ids"], max_length=50, num_return_sequences=1, do_sample=False)

    # Decode and return the result
    print(tokenizer1.decode(outputs[0], skip_special_tokens=True))
    return tokenizer1.decode(outputs[0], skip_special_tokens=True)

In [ ]:
from langdetect import detect  # For language detection
from deep_translator import GoogleTranslator  # For language translation
from gtts import gTTS  # For text-to-speech
import gradio as gr  # For creating the interface

In [ ]:
def classify_real_time_image():
    # Initialize the camera
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

    print("Press 'c' to capture an image for classification, or 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Show the camera feed
        cv2.imshow("Camera Feed", frame)

        # Wait for key press
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):  # 'c' to capture
            # Convert the frame to PIL Image
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            # Perform classification
            results = pipe(image)
            print("Predicted class:", results[0]['label'])
        elif key == ord('q'):  # 'q' to quit
            break

    # Release the camera and close OpenCV window
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
!pip install transformers -U -q

In [ ]:
! pip install sentencepiece

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [ ]:
model_ta = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")



config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
tokenizer_ta = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

In [ ]:
def english_to_tamil(text):
  model_inputs = tokenizer_ta(text,return_tensors="pt")
  generated_tokens = model_ta.generate(**model_inputs,
    forced_bos_token_id= tokenizer_ta.lang_code_to_id["ta_IN"])
  return tokenizer_ta.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [ ]:
translation = english_to_tamil("I love read books and gain insights out of it")
print(translation)

நான் புத்தகங்களைப் படிக்க விரும்புகிறேன், அவற்றில் இருந்து கருத்துக்களை பெறுகிறேன்.


In [ ]:
model_telugu = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")

In [ ]:
tokenizer_telugu = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


In [ ]:
def english_to_telugu(text):
  model_inputs = tokenizer_telugu(text,return_tensors="pt")
  generated_tokens = model_telugu.generate(**model_inputs,
    forced_bos_token_id= tokenizer_telugu.lang_code_to_id["te_IN"])
  return tokenizer_telugu.batch_decode(generated_tokens, skip_special_tokens=True)[0]

In [ ]:
translation_telugu = english_to_telugu("I love read books and gain insights out of it")
print(translation_telugu)

నేనుఇక్కడనుండివిస్తారనిట్టూర్పునిపొందడానికిప్రేమ номలు చదవండి


In [ ]:
def english_to_malayalam(text):
  model_inputs = tokenizer_telugu(text,return_tensors="pt")
  generated_tokens = model_telugu.generate(**model_inputs,
    forced_bos_token_id= tokenizer_telugu.lang_code_to_id["ml_IN"])
  return tokenizer_telugu.batch_decode(generated_tokens, skip_special_tokens=True)[0]

In [ ]:
translation_malayalam = english_to_malayalam("I love read books and gain insights out of it")
print(translation_malayalam)

ഞാന് പുസ്തകങ്ങള് വായിക്കാന് ഇഷ്ടപ്പെടുന്നു, അതില് നിന്ന് ഉള് ക്കാഴ്ചകള് നേടും.


In [ ]:
def english_to_hindi(text):
  model_inputs = tokenizer_telugu(text,return_tensors="pt")
  generated_tokens = model_telugu.generate(**model_inputs,
    forced_bos_token_id= tokenizer_telugu.lang_code_to_id["hi_IN"])
  return tokenizer_telugu.batch_decode(generated_tokens, skip_special_tokens=True)[0]

In [ ]:
translation_hindi = english_to_hindi("I love read books and gain insights out of it")
print(translation_hindi)

मैं किताबें पढ़ना पसंद करता हूं और इससे अंतर्दृष्टि प्राप्त करता हूं।


In [ ]:
def multilingual_tts_runtime(text):
    """
    Ask for the target language at runtime, process a predefined text, and display the output.
    """
    # Predefined input text
    input_text = text  # Example input text

    # Get target language from the user
    target_language = input("Enter the target language code (e.g., 'en' for English): ").strip()

    if target_language == "ta":
        translated_text = english_to_tamil(input_text)
    elif target_language == "hi":
        translated_text = english_to_hindi(input_text)
    elif target_language == "ml":
        translated_text = english_to_malayalam(input_text)
    elif target_language == "te":
        translated_text = english_to_telugu(input_text)
    else:
        print("Invalid target language. Please enter a valid language code.")
        return

    # Import necessary libraries
    import gradio as gr
    from deep_translator import GoogleTranslator
    from gtts import gTTS
    from langdetect import detect

    # Inner function for processing
    def process_translation():
        try:
            # Step 1: Detect the language of the text
            detected_lang = detect(input_text)
            print(f"Detected Language: {detected_lang}")

            # Step 2: Translate the text to the target language

            print(f"Translated Text: {translated_text}")

            # Step 3: Convert the translated text to speech using gTTS
            tts = gTTS(translated_text, lang=target_language)
            output_file = "output.mp3"
            tts.save(output_file)

            # Return results
            return detected_lang, translated_text, output_file

        except Exception as e:
            return "Error", f"An error occurred: {str(e)}", None

    # Wrapper for Gradio output
    def gradio_interface():
        detected_lang, translated_text, audio_file = process_translation()
        return (
            f"Detected Language: {detected_lang}\nTranslated Text: {translated_text}",
            audio_file,
        )

    # Create Gradio app
    interface = gr.Interface(
        fn=lambda: gradio_interface(),  # Function without user-provided inputs
        inputs=None,  # No user inputs required
        outputs=[
            gr.Textbox(label="Translation Details"),  # Show detected language and translation
            gr.Audio(label="Generated Speech"),  # Play the audio file
        ],
        title="Multilingual Text-to-Speech Generator",
        description="This example takes predefined input and displays the results.",
    )

    # Launch the Gradio interface
    interface.launch()



In [ ]:
def classify_user_image(image_path):
    # Load the image
    image = Image.open(image_path).convert("RGB")
    # Perform classification
    results = pipe(image)

    word = results[0]['label']
    description = get_brief_description(word)
    # Run the function
    multilingual_tts_runtime(description)

# Example usage
if __name__ == "__main__":
    print("Select an option:")
    print("1: Classify real-time image from camera")
    print("2: Classify image from file")
    choice = input("Enter your choice (1/2): ")

    if choice == "1":
        classify_real_time_image()
    elif choice == "2":
        image_path = input("Enter the path to the image: ")
        classify_user_image(image_path)
    else:
        print("Invalid choice!")

Select an option:
1: Classify real-time image from camera
2: Classify image from file
Enter your choice (1/2): 2
Enter the path to the image: /content/peacock.jpg
Enter the target language code (e.g., 'en' for English): hi
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6e718453b9608781b5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.1 MB/s eta 0:00:00


In [ ]:
import sacrebleu

# Reference translations (list of strings)
references = ["நான் புத்தகங்களைப் படிக்க விரும்புகிறேன், அதிலிருந்து நுண்ணறிவைப் பெறுகிறேன்"]

# Candidate translation (string)
candidate = "நான் புத்தகங்களைப் படிக்க விரும்புகிறேன், அவற்றில் இருந்து கருத்துக்களை பெறுகிறேன்."

# Compute BLEU score
bleu = sacrebleu.corpus_bleu([candidate], [references])
print(f"BLEU score: {bleu.score}")


BLEU score: 41.11336169005198


In [ ]:
import sacrebleu

# Reference translations (list of strings)
references = ["मुझे किताबें पढ़ना और उससे अंतर्दृष्टि प्राप्त करना पसंद है"]

# Candidate translation (string)
candidate = "मैं किताबें पढ़ना पसंद करता हूं और इससे अंतर्दृष्टि प्राप्त करता हूं"

# Compute BLEU score
bleu = sacrebleu.corpus_bleu([candidate], [references])
print(f"BLEU score: {bleu.score}")


BLEU score: 10.600313379512592


In [ ]:
import sacrebleu

# Reference translations (list of strings)
references = ["నేను పుస్తకాలు చదవడం మరియు దాని నుండి అంతర్దృష్టిని పొందడం చాలా ఇష్టం"]

# Candidate translation (string)
candidate = "నేనుఇక్కడనుండివిస్తారనిట్టూర్పునిపొందడానికిప్రేమ номలు చదవండి"

# Compute BLEU score
bleu = sacrebleu.corpus_bleu([candidate], [references])
print(f"BLEU score: {bleu.score}")


BLEU score: 0.0


In [ ]:
import sacrebleu

# Reference translations (list of strings)
references = ["എനിക്ക് പുസ്തകങ്ങൾ വായിക്കാനും അതിൽ നിന്ന് ഉൾക്കാഴ്ച നേടാനും ഇഷ്ടമാണ്"]

# Candidate translation (string)
candidate = "ഞാന് പുസ്തകങ്ങള് വായിക്കാന് ഇഷ്ടപ്പെടുന്നു, അതില് നിന്ന് ഉള് ക്കാഴ്ചകള് നേടും."

# Compute BLEU score
bleu = sacrebleu.corpus_bleu([candidate], [references])
print(f"BLEU score: {bleu.score}")


BLEU score: 3.7477767366779213


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.translate.meteor_score import meteor_score

# Reference translation (tokenized as a list of words)
reference = ["நான்", "புத்தகங்களைப்", "படிக்க", "விரும்புகிறேன்", ",", "அதிலிருந்து", "நுண்ணறிவைப்", "பெறுகிறேன்"]

# Candidate translation (tokenized as a list of words)
candidate = ["நான்", "புத்தகங்களைப்", "படிக்க", "விரும்புகிறேன்", ",", "அவற்றில்", "இருந்து", "கருத்துக்களை", "பெறுகிறேன்", "."]

# Compute METEOR score
score = meteor_score([reference], candidate)
print(f"METEOR score: {score}")


METEOR score: 0.7181571815718157


In [ ]:
from nltk.translate.meteor_score import meteor_score

# Reference translation (tokenized as a list of words)
reference = ["मुझे", "किताबें", "पढ़ना", "और", "उससे", "अंतर्दृष्टि", "प्राप्त", "करना", "पसंद", "है"]

# Candidate translation (tokenized as a list of words)
candidate = ["मैं", "किताबें", "पढ़ना", "पसंद", "करता", "हूं", "और", "इससे", "अंतर्दृष्टि", "प्राप्त", "करता", "हूं"]

# Compute METEOR score
score = meteor_score([reference], candidate)
print(f"METEOR score: {score}")


METEOR score: 0.5010893246187365


In [ ]:
from nltk.translate.meteor_score import meteor_score

# Reference translation (tokenized as a list of words)
reference = ["నేను", "పుస్తకాలు", "చదవడం", "మరియు", "దాని", "నుండి", "అంతర్దృష్టిని", "పొందడం", "చాలా", "ఇష్టం"]

# Candidate translation (tokenized as a list of words)
candidate = ["నేనుఇక్కడ", "నుండి", "విస్తారనిట్టూర్పునిపొందడానికిప్రేమ", "номలు", "చదవండి"]

# Compute METEOR score
score = meteor_score([reference], candidate)
print(f"METEOR score: {score}")


METEOR score: 0.052631578947368425


In [ ]:
from nltk.translate.meteor_score import meteor_score

# Reference translation (tokenized as a list of words)
reference = ["എനിക്ക്", "പുസ്തകങ്ങൾ", "വായിക്കാനും", "അതിൽ", "നിന്ന്", "ഉൾക്കാഴ്ച", "നേടാനും", "ഇഷ്ടമാണ്"]

# Candidate translation (tokenized as a list of words)
candidate = ["ഞാന്", "പുസ്തകങ്ങള്", "വായിക്കാന്", "ഇഷ്ടപ്പെടുന്നു", ",", "അതില്", "നിന്ന്", "ഉള്", "ക്കാഴ്ചകള്", "നേടും", "."]

# Compute METEOR score
score = meteor_score([reference], candidate)
print(f"METEOR score: {score}")


METEOR score: 0.06024096385542169
